# Predicting Apartment Prices in Russia

In this project, I'll build a machine-learning model to predict the prices of apartments in Russia. The dataset for this project is obtained from [Daniilak on Kaggle](https://www.kaggle.com/datasets/mrdaniilak/russia-real-estate-2021). The documentation (or description) for the dataset is also available on the same Kaggle page.

I'll follow the common machine learning workflow of:
- Prepare data, which in turn has the following steps:
  - Import data
  - Explore data
  - Split data
- Build model
- Communicate results

In [1]:
# Import libraries
import pandas as pd

## Prepare data

### Import

In [2]:
df = pd.read_csv('datasets/russia-real-estate-dataset.csv')

In [3]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 1 columns):
date;price;level;levels;rooms;area;kitchen_area;geo_lat;geo_lon;building_type;object_type;postal_code;street_id;id_region;house_id    object
dtypes: object(1)
memory usage: 86.7+ MB
None


,date;price;level;levels;rooms;area;kitchen_area;geo_lat;geo_lon;building_type;object_type;postal_code;street_id;id_region;house_id
0,2021-01-01;2451300;15;31;1;30.3;0;56.7801124;6...
1,2021-01-01;1450000;5;5;1;33;6;44.6081542;40.13...
2,2021-01-01;10700000;4;13;3;85;12;55.5400601;37...
3,2021-01-01;3100000;3;5;3;82;9;44.6081542;40.13...
4,2021-01-01;2500000;2;3;1;30;9;44.7386846;37.71...


You see in the output above that the data is really messy. It's just a 1 by 1 data frame. But a closer examination of the column title and the row entries shows something interesting: What should have been different features (or columns) are lumped into one, separated by semi-colons.

How do you separate these features to stand alone?

In [4]:
columns = df.columns.str.split(';')[0]
print(columns)

['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']


In [5]:
df = df[df.columns.to_list()[0]].str.split(';', expand=True)

In [6]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 15 columns):
0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
dtypes: object(15)
memory usage: 1.3+ GB
None


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2021-01-01,2451300,15,31,1,30.3,0,56.7801124,60.6993548,0,2,620000,,66,1632918
1,2021-01-01,1450000,5,5,1,33,6,44.6081542,40.1383814,0,0,385000,,1,
2,2021-01-01,10700000,4,13,3,85,12,55.5400601,37.7251124,3,0,142701,242543,50,681306
3,2021-01-01,3100000,3,5,3,82,9,44.6081542,40.1383814,0,0,385000,,1,
4,2021-01-01,2500000,2,3,1,30,9,44.7386846,37.7136681,3,2,353960,439378,23,1730985


The new data frame now has 15 features, but the column headings are not descriptive enough. Let me fix that.

In [8]:
df.columns = columns
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 15 columns):
date             object
price            object
level            object
levels           object
rooms            object
area             object
kitchen_area     object
geo_lat          object
geo_lon          object
building_type    object
object_type      object
postal_code      object
street_id        object
id_region        object
house_id         object
dtypes: object(15)
memory usage: 1.3+ GB
None


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0,56.7801124,60.6993548,0,2,620000,,66,1632918
1,2021-01-01,1450000,5,5,1,33,6,44.6081542,40.1383814,0,0,385000,,1,
2,2021-01-01,10700000,4,13,3,85,12,55.5400601,37.7251124,3,0,142701,242543,50,681306
3,2021-01-01,3100000,3,5,3,82,9,44.6081542,40.1383814,0,0,385000,,1,
4,2021-01-01,2500000,2,3,1,30,9,44.7386846,37.7136681,3,2,353960,439378,23,1730985


Let me save this new data frame as a CSV file, so I don't have to repeat the tedious process of splitting the column. Then I'll import the new CSV file and complete the data wrangling using a function.

In [9]:
df.to_csv('datasets/russia-real-estate-dataset-clean.csv', index=False)

Some data cleaning operations that'll be performed by the `wrangle` function are:
- Import data
- Turn some numerical features into categorical columns

Columns like **id_region**, **building_type** and **object_type** need to be recast into categorical features. From the description of the **id_region** column on the dataset Kaggle page and this [Wikipedia page on region codes in Russia](https://en.wikipedia.org/wiki/Vehicle_registration_plates_of_Russia#Regional_codes), I arrive at the following dictionary for mapping the numbers to regions:

In [3]:
region_id_dict = {
    1: 'Adygea',
    2: 'Bashkortostan',
    3: 'Buryatia',
    4: 'Altai Republic',
    5: 'Dagestan',
    6: 'Ingushetia',
    7: 'Kabardino-Balkar',
    8: 'Kalmykia',
    9: 'Karachay-Cherkess',
    10: 'Karelia',
    11: 'Komi',
    12: 'Mari El',
    13: 'Mordovia',
    14: 'Sakha',
    15: 'North Ossetia–Alania',
    16: 'Tatarstan',
    17: 'Tuva',
    18: 'Udmurt',
    19: 'Khakassia',
    20: 'Chechenya', # Chose 20 because df has no entry for 95, Chechen and Chechenya the same
    21: 'Chuvash',
    22: 'Altai Krai',
    23: 'Krasnodar',
    24: 'Krasnoyarsk',
    25: 'Primorsky',
    26: 'Stavropol',
    27: 'Khabarovsk',
    28: 'Amur',
    29: 'Arkhangelsk',
    30: 'Astrakhan',
    31: 'Belgorod',
    32: 'Bryansk',
    33: 'Vladimir',
    34: 'Volgograd',
    35: 'Vologda',
    36: 'Voronezh',
    37: 'Ivanovo',
    38: 'Irkutsk',
    39: 'Kaliningrad',
    40: 'Kaluga',
    41: 'Kamchatka',
    42: 'Kemerovo',
    43: 'Kirov',
    44: 'Kostroma',
    45: 'Kurgan',
    46: 'Kursk',
    47: 'Leningrad',
    48: 'Lipetsk',
    49: 'Magadan',
    50: 'Moscow Oblast',
    51: 'Murmansk',
    52: 'Nizhny Novgorod',
    53: 'Novgorod',
    54: 'Novosibirsk',
    55: 'Omsk',
    56: 'Orenburg',
    57: 'Oryol',
    58: 'Penza',
    59: 'Perm',
    60: 'Pskov',
    61: 'Rostov',
    62: 'Ryazan',
    63: 'Samara',
    64: 'Saratov',
    65: 'Sakhalin',
    66: 'Sverdlovsk',
    67: 'Smolensk',
    68: 'Tambov',
    69: 'Tver',
    70: 'Tomsk',
    71: 'Tula',
    72: 'Tyumen',
    73: 'Ulyanovsk',
    74: 'Chelyabinsk',
    75: 'Zabaykalsky',
    76: 'Yaroslavl',
    77: 'Moscow',
    78: 'St. Petersburg',
    79: 'Jewish Autonomous Oblast',
    83: 'Nenets',
    86: 'Khanty-Mansi',
    87: 'Chukotka',
    88: 'Evenk',
    89: 'Yamalo-Nenets',
    91: 'Kaliningrad',
    92: 'Sevastopol'
}

For **building_type** and **object_type** columns, the dataset documentation is enough to generate dictionaries for their mapping to categorical features. The following dictionaries are used respectively:

In [ ]:
building_type_dict = {
    0: "Don't know",
    1: "Other",
    2: "Panel",
    3: "Monolithic",
    4: "Brick",
    5: "Blocky",
    6: "Wooden"
}

object_type_dict = {
    0: 'Secondary real estate market',
    2: 'New building'
}

In [ ]:
def wrangle(filepath):
    '''Imports the CSV file holding the data and cleans the data
    
    Arg:
        @filepath: path to the CSV file
        
    Return:
        A clean dataframe
    '''
    pd.read_csv(filepath)

In [2]:
df = pd.read_csv('datasets/russia-real-estate-dataset-clean.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 15 columns):
date             object
price            int64
level            int64
levels           int64
rooms            int64
area             float64
kitchen_area     float64
geo_lat          float64
geo_lon          float64
building_type    int64
object_type      int64
postal_code      float64
street_id        float64
id_region        int64
house_id         float64
dtypes: float64(7), int64(7), object(1)
memory usage: 1.3+ GB
None


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


I'll use a function to complete the rest of the data wrangling process. Almost all the features of this data frame are numerical, which is not descriptive enough for some of these features (e.g. what does a `building_type` of `0` mean?). So, the first thing the wrangle function will do is to replace some of these numerical data with descriptive text data.